In [1]:
from utils.utils import build_market_image, prepare_forecast_data
# from environments.live_environments import LiveTradingEnv
from utils.clients import CoinbaseClient
from Keys import *
# from utils.mappings import oanda_time_map
from utils.reward_functions import sharpe_reward_function
import pandas as pd
from json import dumps

import time
import datetime



In [2]:
# from neuralforecast.core import NeuralForecast
# forecasting_model=NeuralForecast.load('MultiHeadForecastingModel/')
# agent_dir='Agent/final_checkpoints/policies/default_policy'
# agent= Policy.from_checkpoint(agent_dir)

In [3]:
base_asset='DOGE'
quote_asset='USDC'
test_net=False
time_frame='1h'
product_type='FUTURE'
futures_target='DOG-29NOV24-CDE'
exchange='coinbase'
trade_target='/'.join([base_asset,quote_asset]) if product_type=='SPOT' else  futures_target

positions = [-1,0,1]
trade_target,product_type

('DOG-29NOV24-CDE', 'FUTURE')

In [4]:
trade_target

'DOG-29NOV24-CDE'

In [5]:
client=CoinbaseClient(api_key=coinbase_api_key,
                    api_secret=coinbase_api_secret,
                
                    paper=test_net,
                    symbol=trade_target,
                    time_frame=time_frame,
                    product_type=product_type

                    )
client.trade_client.base_url,client.base_asset,client.quote_asset,client.symbol,client.time_frame



['DOG', '29NOV24', 'CDE']


('api.coinbase.com', 'DOG-29NOV24-CDE', 'USD', 'DOG-29NOV24-CDE', '1h')

In [6]:
product_frame=client.get_products(product_type='FUTURE')
product_frame

,product_id,price,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,base_max_size,base_name,quote_name,...,group_description,contract_expiry_timezone,group_short_description,contract_expiry_type,contract_display_name,time_to_expiry_ms,contract_expiry_name,twenty_four_by_seven,0,perpetual_details
0,BIT-29NOV24-CDE,88435,1,0.01,0,100000000,1,100000000,,US Dollar,...,Nano Bitcoin Futures,Europe/London,Nano BTC,EXPIRING,BTC 29 NOV 24,1438920047,2024-11,False,NaN,NaN
1,BTC-USD,87771.76,0.00000001,0.01,1,150000000,0.00000001,3400,Bitcoin,US Dollar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BTC-USDC,87771.76,0.00000001,0.01,1,150000000,0.00000001,3400,Bitcoin,USDC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ETH-PERP-INTX,3236.82,0.0001,0.01,10,50000000,0.0001,50000,,USDC,...,,,,PERPETUAL,ETH PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
4,BTC-PERP-INTX,87801.5,0.0001,0.1,10,50000000,0.0001,3000,,USDC,...,,,,PERPETUAL,BTC PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,ET-28OCT22-CDE,,1,0.5,0,100000000,1,100000000,,US Dollar,...,Nano Ether Futures,,,EXPIRING,ETH 28 OCT 22,-64487879952,,False,NaN,NaN
897,LC-25OCT24-CDE,,1,0.01,0,100000000,1,100000000,,US Dollar,...,Litecoin Futures,Europe/London,Litecoin Futures,EXPIRING,LTC 25 OCT 24,-1588679952,2024-10,False,NaN,NaN
898,BIT-26JAN24-CDE,,1,0.01,0,100000000,1,100000000,,US Dollar,...,Nano Bitcoin Futures,Europe/London,Nano BTC,EXPIRING,BTC 26 JAN 24,-25172279952,,False,NaN,NaN
899,BCH-26JUL24-CDE,,1,0.01,0,100000000,1,100000000,,US Dollar,...,Bitcoin Cash Futures,Europe/London,Bitcoin Cash Futures,EXPIRING,BCH 26 JUL 24,-9451079952,July 2024,False,NaN,NaN


In [7]:
product_frame['contract_root_unit']=product_frame['contract_root_unit'].astype(str)
futures_frame=product_frame[product_frame['contract_expiry_type']=='PERPETUAL'].copy()
# futures_frame=futures_frame[futures_frame['contract_root_unit']=='DOGE'].copy()
futures_frame['contract_expiry']=pd.to_datetime(futures_frame['contract_expiry'])
futures_frame=futures_frame.sort_values('contract_expiry',ascending=True)

In [8]:
futures_frame.reset_index(drop=True,inplace=True)
futures_frame

,product_id,price,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,base_max_size,base_name,quote_name,...,group_description,contract_expiry_timezone,group_short_description,contract_expiry_type,contract_display_name,time_to_expiry_ms,contract_expiry_name,twenty_four_by_seven,0,perpetual_details
0,ETH-PERP-INTX,3236.82,0.0001,0.01,10,50000000,0.0001,50000,,USDC,...,,,,PERPETUAL,ETH PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
1,BTC-PERP-INTX,87801.5,0.0001,0.1,10,50000000,0.0001,3000,,USDC,...,,,,PERPETUAL,BTC PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
2,SOL-PERP-INTX,211.181,0.001,0.001,10,50000000,0.001,3000000,,USDC,...,,,,PERPETUAL,SOL PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
3,DOGE-PERP-INTX,0.37368,1,0.00001,10,50000000,1,914857282.2639668211758966,,USDC,...,,,,PERPETUAL,DOGE PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
4,XRP-PERP-INTX,0.7291,1,0.0001,10,50000000,1,150000000,,USDC,...,,,,PERPETUAL,XRP PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,IMX-PERP-INTX,1.3545,0.1,0.0001,10,50000000,0.1,1000000000000000000,,USDC,...,,,,PERPETUAL,IMX PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
92,ETHFI-PERP-INTX,1.7,0.1,0.001,10,50000000,0.1,1000000000000000000,,USDC,...,,,,PERPETUAL,ETHFI PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
93,ZRO-PERP-INTX,3.827,0.1,0.001,10,50000000,0.1,1000000000000000000,,USDC,...,,,,PERPETUAL,ZRO PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '0.0000..."
94,NEO-PERP-INTX,11.488,0.1,0.001,10,50000000,0.1,1000000000000000000,,USDC,...,,,,PERPETUAL,NEO PERPETUAL,0,,False,NaN,"{'open_interest': '0', 'funding_rate': '-0.000..."


In [9]:
future_ids=futures_frame['product_id'].unique()
future_ids

array(['ETH-PERP-INTX', 'BTC-PERP-INTX', 'SOL-PERP-INTX',
       'DOGE-PERP-INTX', 'XRP-PERP-INTX', 'XLM-PERP-INTX',
       'SUI-PERP-INTX', 'AVAX-PERP-INTX', 'NEAR-PERP-INTX',
       '1000SHIB-PERP-INTX', 'APT-PERP-INTX', 'WIF-PERP-INTX',
       'RUNE-PERP-INTX', '1000PEPE-PERP-INTX', 'ADA-PERP-INTX',
       'FIL-PERP-INTX', 'LTC-PERP-INTX', 'BCH-PERP-INTX',
       '1000BONK-PERP-INTX', 'DEGEN-PERP-INTX', 'SEI-PERP-INTX',
       'DOT-PERP-INTX', 'MEW-PERP-INTX', '1000SATS-PERP-INTX',
       'ENA-PERP-INTX', 'OP-PERP-INTX', 'INJ-PERP-INTX', 'WLD-PERP-INTX',
       '1000FLOKI-PERP-INTX', 'BB-PERP-INTX', 'JUP-PERP-INTX',
       'FTM-PERP-INTX', 'LINK-PERP-INTX', 'OM-PERP-INTX',
       'SAGA-PERP-INTX', 'UNI-PERP-INTX', 'BLUR-PERP-INTX',
       'BOME-PERP-INTX', 'HBAR-PERP-INTX', 'ZK-PERP-INTX',
       'ETC-PERP-INTX', 'FET-PERP-INTX', 'NOT-PERP-INTX',
       'TNSR-PERP-INTX', 'EOS-PERP-INTX', 'RENDER-PERP-INTX',
       'AAVE-PERP-INTX', 'CFX-PERP-INTX', 'PEOPLE-PERP-INTX',
       'ARKM-P

In [10]:
expire_ranges=futures_frame['contract_expiry'].unique()

In [11]:
futures_frame[['product_id','contract_expiry']]
futures_frame['trading_start']=futures_frame['contract_expiry'].shift(1)
futures_frame[['product_id','trading_start','contract_expiry']]
futures_frame['trading_start']=futures_frame['trading_start'].fillna(futures_frame['contract_expiry']-pd.Timedelta(days=28))
futures_frame=futures_frame[['product_id','trading_start','contract_expiry']]
futures_frame

,product_id,trading_start,contract_expiry
0,ETH-PERP-INTX,NaT,NaT
1,BTC-PERP-INTX,NaT,NaT
2,SOL-PERP-INTX,NaT,NaT
3,DOGE-PERP-INTX,NaT,NaT
4,XRP-PERP-INTX,NaT,NaT
...,...,...,...
91,IMX-PERP-INTX,NaT,NaT
92,ETHFI-PERP-INTX,NaT,NaT
93,ZRO-PERP-INTX,NaT,NaT
94,NEO-PERP-INTX,NaT,NaT


In [12]:
target_list=['DOGE','SHIB','ETH','BTC','SUI','LTC']
from tqdm.autonotebook import tqdm

In [13]:
start_date,end_date='2024-05-01','2024-11-12'
start_date=pd.Timestamp(start_date,tz= 'utc')
end_date=pd.Timestamp(end_date,tz= 'utc')

for idx in tqdm(future_ids,leave=True):
    symbol_base=idx.split('-')[0]
    symbol_name=symbol_base+'USD_Futures'
    if symbol_base in target_list:
        data=client.bulk_download(idx,start_date,end_date,limit=300,symbol_name=symbol_name)
        data.to_pickle(f'data_futures/{exchange}-{symbol_name}-{time_frame}.pkl')


  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [14]:
from IPython.display import display

In [15]:
def download_futures_data(client,symbol_base='DOGE'):
    product_frame=client.get_products(product_type='FUTURE')
    product_frame['contract_root_unit']=product_frame['contract_root_unit'].astype(str)
    futures_frame=product_frame[product_frame['contract_root_unit']==symbol_base.upper()].copy()
    
    futures_frame['contract_expiry']=pd.to_datetime(futures_frame['contract_expiry'])
    futures_frame=futures_frame[futures_frame['contract_expiry_type']=='EXPIRING']
    futures_frame=futures_frame[['product_id','contract_expiry']].reset_index(drop=True)
    ascend=False
    futures_frame=futures_frame.sort_values('contract_expiry',ascending=ascend)
    futures_frame['trading_start']=futures_frame['contract_expiry'].shift(-1 if ascend==False else 1)
    futures_frame=futures_frame[['product_id','trading_start','contract_expiry']]
    futures_frame['trading_start']=futures_frame['trading_start'].fillna(futures_frame['contract_expiry']-pd.Timedelta(days=28))
    # display(futures_frame)
    print(pd.Timestamp.now(tz='utc').strftime('%Y-%m-%d %H:%M'))
    futures_frame=futures_frame[futures_frame['trading_start']<pd.Timestamp.now(tz='utc')]

    
    display(futures_frame)

    futures_data=[]
    for i,row in futures_frame.iterrows():
        symbol=row['product_id']
        start_date=row['trading_start']
        end_date=row['contract_expiry']
        
        data=bulk_download(client,symbol,start_date,end_date,limit=300)
        futures_data.append(data)
        time.sleep(1)
        print(symbol_base,start_date,end_date,len(data))

        
    futures_data=pd.concat(futures_data)
    futures_data.sort_values('date_close',inplace=True)
    futures_data=futures_data.drop_duplicates()
    return futures_data

In [16]:
start=pd.Timestamp('2024-08-30 15:00:00+00:00')
end=start+pd.Timedelta(hours=300)
start.timestamp(),end.timestamp()

(1725030000.0, 1726110000.0)

In [17]:
download_futures_data(client,'DOGE')

2024-11-13 00:05


,product_id,trading_start,contract_expiry
0,DOG-29NOV24-CDE,2024-10-25 15:00:00+00:00,2024-11-29 16:00:00+00:00
5,DOG-25OCT24-CDE,2024-09-27 15:00:00+00:00,2024-10-25 15:00:00+00:00
7,DOG-27SEP24-CDE,2024-08-30 15:00:00+00:00,2024-09-27 15:00:00+00:00
2,DOG-30AUG24-CDE,2024-07-26 15:00:00+00:00,2024-08-30 15:00:00+00:00
8,DOG-26JUL24-CDE,2024-06-28 15:00:00+00:00,2024-07-26 15:00:00+00:00
6,DOG-28JUN24-CDE,2024-05-31 15:00:00+00:00,2024-06-28 15:00:00+00:00
3,DOG-31MAY24-CDE,2024-05-03 15:00:00+00:00,2024-05-31 15:00:00+00:00


NameError: name 'bulk_download' is not defined

In [ ]:
account=client.account()
account



In [ ]:
start_time = datetime.datetime(month=11,day=1,year=2024)
end_time = datetime.datetime(month=11,day=12,year=2024)
pd.date_range(start=start_time,end=end_time,freq='1h')
def get_time_ranges(start_time,end_time,freq='1h'):
    return pd.date_range(start=start_time,end=end_time,freq=freq)


In [ ]:
def download_futures(start_date='2021-01-01',end_date='2021-10-01',limit=350):
    data=client.get_historical_data(symbol=futures_target,start_date=start_date,end_date=end_date,limit=limit)
    data.to_csv('data/futures.csv',index=False)
    return data

In [ ]:
product_frame=client.get_products()
product_frame

In [ ]:
product_frame[product_frame['contract_code']=='DOG']

In [ ]:
product_details=product_frame['future_product_details'].apply(lambda x: pd.Series(x))
product_details
# summary=summary['bproducts']
# summary.pop('intraday_margin_window_measure')
# summary.pop('overnight_margin_window_measure')


In [ ]:
positions=client.trade_client.list_futures_positions()

In [ ]:
positions

In [ ]:

product_frame=pd.DataFrame(products['products'])
product_frame=product_frame[product_frame['view_only']==False]

# product_frame=product_frame[product_frame['trading_disabled']==False]

product_frame

In [ ]:
trade_info

In [ ]:
# products=client.trade_client.get_products()
now=datetime.datetime.now()
start_time=now-datetime.timedelta(hours=input_size*2)

int(now.timestamp())

In [ ]:
fills=client.get_orders()
fills

In [ ]:
time.sleep(10)

In [ ]:
client.sell('BTC-USDC',base_size='.000014')

In [ ]:
time.sleep(10)

In [ ]:
fills=client.get_orders()
fills